In [160]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np


In [161]:
pd.set_option('display.max_rows', 500)

In [162]:
def read_files_in_directory(directory):
    file_paths = []
    for root, _, files in os.walk(directory):

        for file in files:
            if file[-3:] == "tsv":
                file_path = os.path.join(root, file)
                file_paths.append(file_path)
    return file_paths


In [163]:
directory_path = "data/Protein_Expression"
files = read_files_in_directory(directory_path)
len(files)

7906

In [164]:
def get_case_id_from_filename(file):
    index = file.find("TCGA")
    return file[index:index+12]

In [165]:
def extract_protein_expression(file):
    df = pd.read_csv(file, sep='\t')
    subset_df = df[['protein_expression', 'AGID']]
    transposed_df = subset_df.set_index('AGID').T
    transposed_df = transposed_df.reset_index(drop=True)
    transposed_df.insert(0, 'case_submitter_id', [get_case_id_from_filename(file)])
    return transposed_df

In [166]:
def merge_cases(files,extractor):
    merged = extractor(files[0])
    for file in files[1:]:
        to_be_merge = extractor(file)
        merged = pd.concat([to_be_merge, merged], axis=0, ignore_index=True)
    return merged
    

In [167]:
# merged_rppa = merge_cases(files,extract_protein_expression)
# merged_rppa.to_csv("full_rppa_raw.csv",index=False)


In [168]:
merged_rppa = pd.read_csv('full_rppa_raw.csv')
merged_rppa.shape

(7906, 488)

In [169]:
def calculate_column_statistics(df):
    results = pd.DataFrame(index=df.columns, columns=['Mean', 'Variance', 'NumUniqueValues', 'NumNaNs'])
    for column in df.columns:
        if column not in ["case_submitter_id"]:
            results.at[column, 'Mean'] = df[column].mean()
            results.at[column, 'Variance'] = df[column].var()
            results.at[column, 'NumUniqueValues'] = df[column].nunique()
            results.at[column, 'NumNaNs'] = df[column].isna().sum()
    return results

In [170]:
statistics= calculate_column_statistics(merged_rppa)
statistics["NumNaNs"].value_counts()

NumNaNs
406     238
0       216
7252     12
7146     10
5335      4
6964      1
4691      1
1000      1
4237      1
2417      1
6110      1
1415      1
Name: count, dtype: int64

In [171]:
7906*0.2

1581.2

among all 488 proteins, for 20% Missingness, 7906*0.2 = 1581.2, we drop 12 + 10 + 4 + 1 + 1 + 1 + 1 + 1 =31 proteins since too many missing. For other 238 + 1(1000) + 1(1415) preotins consider inputation.

Drop

In [172]:
def extract_rows_by_nan(df, nan_threshold=1581):
    columns = [column for column in df.columns if df[column].isna().sum() >= nan_threshold]
    return columns

In [173]:
empmpty_columns = extract_rows_by_nan(merged_rppa,1581)
merged_rppa = merged_rppa.drop(columns=empmpty_columns)
merged_rppa.shape

(7906, 457)

Inputation

In [174]:
path = "data/clinical.cart.2024-06-25/clinical.tsv"
df = pd.read_csv(path,sep='\t')

In [175]:
df['days_to_death'] = pd.to_numeric(df['days_to_death'], errors='coerce')
df['days_to_last_follow_up'] = pd.to_numeric(df['days_to_last_follow_up'], errors='coerce')

In [176]:
colums = ["ajcc_pathologic_stage","vital_status","days_to_death","days_to_last_follow_up","case_submitter_id","primary_diagnosis"]
deduplicate = df[df.index %2 !=0].sort_values(by='case_submitter_id')

In [177]:
merged_rppa = pd.merge(merged_rppa, deduplicate[colums], on='case_submitter_id', how='inner')
merged_rppa["ajcc_pathologic_stage"].unique()

array(['Stage IV', 'Stage I', "'--", 'Stage III', 'Stage IIB', 'Stage IA',
       'Stage II', 'Stage IIA', 'Stage IIIA', 'Stage IB', 'Stage IIIB',
       'Stage IVA', 'Stage 0', 'Stage IIIC', 'Stage IIC', 'Stage X',
       'Stage IVB', 'Stage IVC', 'Stage IS', 'Not Reported'], dtype=object)

In [178]:
def encode_stage_column(df, column_name):
    # Define mapping dictionary
    stage_mapping = {
        "Stage I": 0,
        "Stage IA": 1,
        "Stage IB": 2,
        "Stage IS": 3,
        "Stage II": 4,
        "Stage IIA": 5,
        "Stage IIB": 6,
        "Stage IIC": 7,
        "Stage III": 8,
        "Stage IIIA": 9,
        "Stage IIIB": 10,
        "Stage IIIC": 11,
        "Stage IV": 12,
        "Stage IVA": 13,
        "Stage IVB": 14,
        "Stage IVC": 15,
        "Stage X": 16,
        "Stage 0":17,
        "'--": -1,
        "Not Reported": -1
        
            }
    
    # Apply mapping to the column
    df[column_name] = df[column_name].map(stage_mapping)
    
    return df

In [179]:
merged_rppa_before_inputation = encode_stage_column(merged_rppa,"ajcc_pathologic_stage")

In [181]:
proteins_only_cols = merged_rppa_before_inputation.columns.drop(["case_submitter_id", 'ajcc_pathologic_stage', 'vital_status', 'days_to_death',
       'days_to_last_follow_up', 'primary_diagnosis'])

In [182]:
statistics= calculate_column_statistics(merged_rppa_before_inputation[proteins_only_cols])
statistics["NumNaNs"].value_counts()

NumNaNs
405     238
0       216
990       1
1414      1
Name: count, dtype: int64

In [183]:
merged_rppa_before_inputation.shape

(7895, 462)

7906-7895 = 11 cases are dropped while merging since lack of clinical information

among proteins that need to be inputated, drop cases if pathologic_stage not reported

In [184]:
def extract_columns_by_nan_num(df, nan_threshold):
    columns = [column for column in df.columns if df[column].isna().sum() == nan_threshold]
    return columns

In [191]:
protein_0f_1414_missing = extract_columns_by_nan_num(merged_rppa_before_inputation,1414)
protein_0f_990_missing = extract_columns_by_nan_num(merged_rppa_before_inputation,990)
protein_0f_405_missing = extract_columns_by_nan_num(merged_rppa_before_inputation,405)


In [195]:
def drop_rows(df, columns_to_check):
    all_nan = df[columns_to_check].isnull().all(axis=1)
    stage_minus_one = df["ajcc_pathologic_stage"] == -1
    to_drop = all_nan & stage_minus_one
    df = df[~to_drop]
    return df


In [196]:
merged_df_no_inputation = drop_rows(merged_rppa_before_inputation,protein_0f_1414_missing)
merged_df_no_inputation = drop_rows(merged_df_no_inputation,protein_0f_990_missing)
merged_df_no_inputation = drop_rows(merged_df_no_inputation,protein_0f_405_missing)

In [197]:
merged_df_no_inputation.shape

(7229, 462)

7895 - 7229 = 666 are dropped , good, most proteins that need inputation have ajcc_pathologic_stage avaliable

Now we inpute AGID00385, the protien that missed in 1414 cases

In [199]:
merged_df_no_inputation["AGID00385"].isna().sum()

943

After drop, it left us 943 cases to inpute

In [200]:
def impute_missing_values(df, group_cols, target_col):

    # Calculate median for each group
    median_values = df.groupby(group_cols)[target_col].median().reset_index()

    # Create a dictionary for easy lookup
    median_dict = median_values.set_index(group_cols)[target_col].to_dict()

    # Define a function to fill missing values using the median_dict
    def fill_missing(row):
        if pd.isna(row[target_col]):
            return median_dict.get(tuple(row[col] for col in group_cols), row[target_col])
        else:
            return row[target_col]

    # Apply the function to fill missing values
    df[target_col] = df.apply(fill_missing, axis=1)
    
    return df

In [201]:
merged_df_no_inputation_test = impute_missing_values(merged_df_no_inputation,["primary_diagnosis","ajcc_pathologic_stage"],protein_0f_1414_missing[0])

In [202]:
merged_df_no_inputation_test[["AGID00385","primary_diagnosis","ajcc_pathologic_stage"]]
mask = (merged_df_no_inputation_test[protein_0f_1414_missing[0]].isna())
merged_df_no_inputation_test[mask].shape

(540, 462)

oh no, these 540 cases come from groups that this protein is all missed and cannot determine the average, considering the ratio 540/7895 and 1/462, we may have to drop this protein.

In [133]:
merged_df_no_inputation = merged_df_no_inputation.drop(protein_0f_1414_missing,axis=1)

In [109]:
protein_0f_990_missing

['AGID00445']

Now we move to AGID00445, which is missing in 990 cases

In [203]:
merged_df_no_inputation["AGID00445"].isna().sum()

794

In [204]:
merged_df_no_inputation_test = impute_missing_values(merged_df_no_inputation,["primary_diagnosis","ajcc_pathologic_stage"],"AGID00445")

In [205]:
merged_df_no_inputation[["AGID00445","primary_diagnosis","ajcc_pathologic_stage"]]
mask = (merged_df_no_inputation["AGID00445"].isna())
merged_df_no_inputation[mask][["AGID00445","primary_diagnosis","ajcc_pathologic_stage"]].shape

(184, 3)

oh no, still left use 184 cases